# Program de clasificare a amprentelor digitale

Acest program utilizează o rețea neuronală antrenată pentru a clasifica amprentele digitale în clasele Arc, Vârtej și Buclă

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import os
from PIL import Image
from torch.utils.data import Dataset
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm

In [17]:
datadir = '/content/drive/MyDrive/Facultate Informatica/Profesor/2024-2025/Sisteme expert si metode biometrice in securitatea informatiei/Curs/ColabMount/DATA/test_arc_images'
files = os.listdir(datadir)
print(files)

['class1_Arc_0030.png', 'class1_Arc_0032.png', 'class1_Arc_0022.png', 'class1_Arc_0010.png', 'class1_Arc_0001.png', 'class1_Arc_0010_v1.png', 'class1_Arc_0010_v2.png', 'class1_Arc_0010_v3.png']


In [18]:
model_file = '/content/drive/MyDrive/Facultate Informatica/Profesor/2024-2025/Sisteme expert si metode biometrice in securitatea informatiei/Curs/ColabMount/Models/fingerprint_classifier.pth'
if (os.path.exists(model_file)):
  print("modelul exista")
else:
  print("modelul nu exista")

modelul exista


## Reteaua neuronala

Această rețea este simplă, dar eficientă pentru sarcini de clasificare a imaginilor de amprente în mai multe categorii (de exemplu, clasificare biometrică). Rețeaua poate fi extinsă sau modificată cu ușurință pentru a se adapta altor dimensiuni de intrare, alte dimensiuni ale embedding-ului sau un număr diferit de clase.

Rețeaua procesează imagini în tonuri de gri (un singur canal) de dimensiune pătrată (implicit 128x128 pixeli) și clasifică fiecare imagine într-una dintre cele num_classes (implicit 3 clase posibile).

Blocul convoluțional conține trei straturi convoluționale, fiecare urmat de o funcție de activare ReLU. După al doilea și al treilea strat convoluțional se aplică pooling (MaxPool2d) pentru reducerea dimensiunii. Dimensiunile canalelor cresc astfel: 1 → 32 → 64 → 128, adaptând progresiv reprezentarea imaginii.

Ultimul strat are 32 de pixeli, un stat relativ mare, datorită structurilor relativ reduse ale amprentelor.

După straturile convoluționale, ieșirea este aplatizată (Flatten). Aceasta este urmată de un strat Linear care generează un spațiu de embedding (implicit 512 dimensiuni), urmat de ReLU. Ultimul strat este un strat linear care generează logits pentru fiecare clasă (nu se aplică sigmoid deoarece se presupune o clasificare multi-clasă).

In [19]:
class FingerprintClassifier(nn.Module):
    def __init__(self, image_size=128, embedding_size=512, num_classes=3):
        super(FingerprintClassifier, self).__init__()
        self.image_size = image_size

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),  # 1 input channel for grayscale
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv_output_size = self._get_conv_output_size()

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.conv_output_size, embedding_size),
            nn.ReLU(),
            nn.Linear(embedding_size, num_classes)  # No sigmoid for multi-class logits
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

    def _get_conv_output_size(self):
        dummy_input = torch.ones(1, 1, self.image_size, self.image_size)  # 1 channel for grayscale
        x = self.conv_layers(dummy_input)
        return x.numel()


## Functia de inferenta

Aceasta functie face urmatoarele:

- initializeaza reteaua neuronala ca o instanta a clasei FingerprintClassifier
- incarca modelul de pe disc
- pune reteaua neuronala in modul de evaluare
- face o lista a tuturor fisierelor din director

Pentru fiecare imagine din lista:
- citeste imaginea
- modifica imaginea adaugand dimensiunea batch
- trece imaginea prin reteaua neuronala
- extrage clasa cea mai probabila

In [12]:
def classify_images_in_directory(test_dir, model_path=model_file):
    # Load model
    model = FingerprintClassifier().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    # Collect image paths
    image_files = [
        os.path.join(test_dir, f)
        for f in os.listdir(test_dir)
        if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif'))
    ]

    if not image_files:
        print("No images found in the directory.")
        return

    # Run classification
    for img_path in sorted(image_files):
        image = Image.open(img_path).convert('L')
        image = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(image)
            predicted_class = output.argmax(dim=1).item()
            print(f"{os.path.basename(img_path)} => {label_map[predicted_class]}")

## Definirea dispozitivului si a funcției de transformare

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Category labels (adjust if more/less classes)
label_map = {
    0: 'class1_Arc',
    1: 'class2_Whorl',
    2: 'class3_Loop'
}

# Image transformation (should match training)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [14]:
classify_images_in_directory(datadir)

class1_Arc_0001.png => class1_Arc
class1_Arc_0010.png => class1_Arc
class1_Arc_0010_v1.png => class1_Arc
class1_Arc_0010_v2.png => class1_Arc
class1_Arc_0010_v3.png => class1_Arc
class1_Arc_0022.png => class1_Arc
class1_Arc_0030.png => class1_Arc
class1_Arc_0032.png => class1_Arc
